# ivi Big Data school

## Pandas

### Получение данных

In [10]:
import json # читаем конфиги

import pandas as pd # гвоздь программы
import numpy as np # никогда не помешает немного numpy
from pyhive import presto # чтобы забрать данные

In [7]:
!mkdir -p ./data
!ls | grep data

data


In [69]:
sql_string = """
SELECT
	cw.ivi_id
    ,op.name as operator_name
	,CASE WHEN c.compilation_id=898989 THEN c.id ELSE cmp.id END as content_id
	,CASE WHEN c.compilation_id=898989 THEN c.title ELSE cmp.title END as content_title
    ,cw.show_duration
FROM 
	groot.content_watch cw
INNER JOIN groot.content c
	on c.id = cw.content_id
INNER JOIN groot.operator op
	on op.id = operator_id
LEFT JOIN groot.compilation cmp
	on cmp.id = c.compilation_id
WHERE
	dt = '2017-08-01'
	AND subsite_id = 353
	AND NOT cw.ivi_id IS NULL
LIMIT 20000
"""

# загружаем данные для подключения из конфига
with open('../presto_config.json') as f:
    config = json.loads(f.read())

cursor = presto.connect(host=config["PRESTO_HOST"],
                        port=config["PRESTO_PORT"],
                        username=config["PRESTO_USER"]).cursor()

cursor.execute(sql_string)
data = cursor.fetchall()
# посмотрим на результат в страшном виде
print '"Сырые" данные:\n{}'.format(data[:3])
# сохраним в csv
with open('./content_watch.csv','w') as f:
    for i in data:
        f.write('\t'.join([
                    str(j.encode('utf-8')) 
                    if (type(j)<>int and not j is None)
                    else str(j) 
                for j in i]) + '\n'
        )

"Сырые" данные:
[[827375963, u'Rostelecom', 9966, u'\u041b\u0443\u043d\u0430', 1400], [890810589, u'LLC TOMTEL', 10140, u'\u0421\u043e\u043b\u0434\u0430\u0442\u044b', 1271], [929088258, u'OJS Moscow city telephone network', 9572, u'\u041c\u0430\u0436\u043e\u0440', 30]]


### Создание pandas DataFrame

Фреймы можно создавать двумя как из csv, так и из итерируемой последовательности "строк" будущей таблицы

In [70]:
col_names = ["user_id", "operator_name", "content_id", "content_title", "show_duration"]
# создаём фрейм из последовательности
df_from_iterable = pd.DataFrame(data, columns=col_names)

# читаем из подготовленного csv
df = pd.read_csv('./content_watch.csv',
                 sep='\t',
                 header=None, # можно не передавать, если в файле есть заголовки столбцов
                 names = col_names)

#смотрим, что получилось
print(df_from_iterable.head(3))
print('-'*10)
df.head(3)

     user_id                      operator_name  content_id content_title  \
0  827375963                         Rostelecom        9966          Луна   
1  890810589                         LLC TOMTEL       10140       Солдаты   
2  929088258  OJS Moscow city telephone network        9572         Мажор   

   show_duration  
0         1400.0  
1         1271.0  
2           30.0  
----------


,user_id,operator_name,content_id,content_title,show_duration
0,827375963,Rostelecom,9966,Луна,1400
1,890810589,LLC TOMTEL,10140,Солдаты,1271
2,929088258,OJS Moscow city telephone network,9572,Мажор,30


### Первый взгляд на данные

вещи, которые пригодятся всегда

In [74]:
print(df.shape) # размерность таблички
print('-'*10)
print(df.columns) # имена колонок
print('-'*10)
print(df.info()) # информация о типах данных
df.describe() # можно передать include=['object'] чтобы посчитать статиcтику по категориальным признакам

(20000, 5)
----------
Index([u'user_id', u'operator_name', u'content_id', u'content_title',
       u'show_duration'],
      dtype='object')
----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 5 columns):
user_id          20000 non-null object
operator_name    20000 non-null object
content_id       20000 non-null object
content_title    20000 non-null object
show_duration    20000 non-null object
dtypes: object(5)
memory usage: 781.3+ KB
None


,user_id,operator_name,content_id,content_title,show_duration
count,20000,20000,20000,20000,20000
unique,10418,576,1163,1155,2901
top,547441836,Rostelecom,7305,Мастер и Маргарита,None
freq,66,5233,1972,1972,2072


In [79]:
# Тип колонки Pandas вычисляет автоматически - можно поменять на более актуальный
df['content_id'] = df['content_id'].astype('object')
df['user_id'] = df['user_id'].astype('object')

#### Доступ к данным

как брать различные срезы данных

In [104]:
# Срез по колонкам 
col_slice = ['user_id', 'content_id']
df[col_slice].head(3)

,user_id,content_id
0,827375963,9966
1,890810589,10140
2,929088258,9572


In [117]:
# loc - индексация, основаная на названиях
# [start:end:step] - можно менять порядок в выдаче
df.loc[4:2:-2,col_slice]
# Индексы строк сейчас имеют целочисленный тип, но так будет не всегда
# df.loc[2:4, [0,2] ] - вызовет TypeError, т.к. для .loc колонки 2 не существует

,user_id,content_id
4,573841374,9983
2,929088258,9572


In [118]:
# для индексации по числовым индексам - iloc
df.iloc[4:2:-2, [2,3]]
# загадка - сколько строк будет в выводе?

,content_id,content_title
4,9983,Метод


### Очистка данных

"Сырые" данные напрямую из источников плохо пригодны для анализа. Очистить и подготовить данные можно с помощью Pаndas

In [94]:
# Почему show_duration имеет тип object?
print (df['show_duration'].value_counts().head(5)) # простая метрика - счётчик элементов. Считаем просмотры контента
# normalize=True - посчитаем доли объектов вместо абсолютных значений
print ("Доля None в колонке show_duration = {}".format(
        df['show_duration'].value_counts(normalize=True).head(1).values[0]))

None    2072
9        481
6        462
60       354
2        303
Name: show_duration, dtype: int64
Доля None в колонке show_duration = 0.1036


In [120]:
# Поменяем значение к колонке с помощью уже изученного loc - аналогично конструкции WHERE в SQL
df.loc[df['show_duration']=='None', 'show_duration'] = 0
# домашнее задание - сделать с помощью  другого метода pandas.DataFarame
df[df['show_duration']=='None'].head(5)

In [139]:
# другой способ получить логический срез
df[df['content_id']==7029].head(3)
# df[df['content_id']==7029]['content_id'] = 7029 - так тоже попробуйте

,user_id,operator_name,content_id,content_title,show_duration
5943,7029,7029,7029,7029,7029
5946,7029,7029,7029,7029,7029
5947,7029,7029,7029,7029,7029


In [141]:
# быстрый способ удалить строки
df = df[df['content_id']!=7029]
df[df['content_id']==7029].head(3)

,user_id,operator_name,content_id,content_title,show_duration


In [ ]:
# Аггрегаты можно вычислять по столбцам